In [1]:
# !echo $RAYCLUSTER_AUTOSCALER_HEAD_SVC_PORT_10001_TCP_ADDR
# !echo $RAYCLUSTER_AUTOSCALER_HEAD_SVC_PORT_10001_TCP_PORT

In [2]:
import sys; sys.path.insert(0, '../..') # add parent folder path where lib folder is

In [3]:
import ray
import time

In [4]:
# from utils import logger

In [6]:
%load_ext autoreload
%autoreload 2

from utils import helper, config, rayer
from ml.models.base.v2.brisk_bagging import BriskBagging

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from ml.models import common



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
rayer.get_global_cluster(num_cpus=4)

In [8]:
!ray status --address='raycluster-autoscaler-head-svc.dev.svc.cluster.local:6379'

======== Autoscaler status: 2022-11-22 12:09:13.951432 ========
Node status
---------------------------------------------------------------
Healthy:
 1 head-group
 1 small-group
Pending:
 (no pending nodes)
Recent failures:
 (no failures)

Resources
---------------------------------------------------------------
Usage:
 0.0/3.0 CPU
 0.00/9.313 GiB memory
 0.00/2.696 GiB object_store_memory

Demands:
 (no resource demands)


In [8]:
# ray.autoscaler.sdk.request_resources(num_cpus=0)

In [9]:
X, y = helper.get_covid_dataset()
X = X.drop(['location'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

X_train_scalar = StandardScaler().fit_transform(X_train.copy())
X_test_scalar = StandardScaler().fit_transform(X_test.copy())


(scheduler +3s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(scheduler +3s) Adding 1 node(s) of type small-group.


In [16]:

brisk_bagging = BriskBagging('brisk_xgb')
brisk_bagging.boosted_round = 100
brisk_bagging.n_trials = 50


In [17]:
brisk_bagging.fetch_model(X_train_scalar, X_test_scalar, y_train, y_test)

2022-11-22T12:12:21CST : INFO : brisk_bagging : __discover_model__ : 94 : Message : brisk_xgb: Starting training for trials:50, n_estimators  30
2022-11-22T12:12:22CST : INFO : brisk_bagging : __discover_model__ : 112 : Message : brisk_xgb: Number of trials: 50
2022-11-22T12:12:22CST : INFO : brisk_bagging : __discover_model__ : 114 : Message : Best trial:40
2022-11-22T12:12:22CST : INFO : brisk_bagging : __discover_model__ : 116 : Message :   Params: 
2022-11-22T12:12:22CST : INFO : brisk_bagging : __discover_model__ : 118 : Message :     n_estimators 10
2022-11-22T12:12:22CST : INFO : brisk_bagging : __discover_model__ : 132 : Message :   test r2 score: 0.7444660486202793
2022-11-22T12:12:22CST : INFO : brisk_bagging : __discover_model__ : 137 : Message : brisk_xgb: Model saved at /tmp/covid_autolearn/brisk_xgb/saved/brisk_xgb_40.pickle


BaggingRegressor()

In [18]:
brisk_bagging.score

[0.9324550152093946, 0.7444660486202793]

(scheduler +3m41s) Resized to 5 CPUs.


In [16]:

slug_xgb = SlugXGBoost('slug_xgb', X_train_scalar, X_test_scalar, y_train, y_test)
slug_xgb.boosted_round = 50
slug_xgb.n_trials = 1000


In [17]:
slug_xgb.fetch_model()

2022-11-17T13:52:14CST : INFO : slug_xgboost : __discover_model__ : 168 : Message : slug_xgb: Starting training for trials:1000, boosted rounds: 50, max depth: 10
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ : 181 : Message : slug_xgb: Number of trials: 1000
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ : 183 : Message : Best trial:611
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ : 185 : Message :   Params: 
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ : 187 : Message :     lambda 0.019399866419368653
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ : 187 : Message :     alpha 0.0020906308336665715
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ : 187 : Message :     eta 0.3727957591519319
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ : 187 : Message :     gamma 0.0033926747144862554
2022-11-17T13:52:44CST : INFO : slug_xgboost : __discover_model__ :

In [36]:

slug_rf = SlugRF('slug_rf', X_train_scalar, X_test_scalar, y_train, y_test)
slug_rf.max_n_estimators = 5000
slug_rf.n_trials = 100
slug_rf.max_depth = 200

In [37]:
best_fit = slug_rf.fetch_model()

2022-11-17T14:56:36CST : INFO : slug_rf : __discover_model__ : 105 : Message : slug_rf: Starting training for trials:100, max depth: 200, max estimators 5000
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 122 : Message : slug_rf: Number of trials: 100
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 124 : Message : Best trial:74
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 126 : Message :   Params: 
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 128 : Message :     n_estimators 62
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 128 : Message :     max_depth 26
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 128 : Message :     min_samples_split 52
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 128 : Message :     min_samples_leaf 34
2022-11-17T14:59:59CST : INFO : slug_rf : __discover_model__ : 142 : Message :   test r2 score: 0.3992977208659255
2022-11-17T14:59:59CST : INFO : 

In [35]:
slug_rf.score

[0.5264056467634264, 0.39565984792823017]